In [11]:
%load_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [13]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS User (
    userID INTEGER PRIMARY KEY AUTOINCREMENT,
    preference TEXT
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [15]:
%%sql
CREATE TABLE IF NOT EXISTS Item (
    ItemID INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    type TEXT NOT NULL,
    author TEXT,
    status TEXT
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [19]:
%%sql
CREATE TABLE IF NOT EXISTS CD (
    ItemID INTEGER PRIMARY KEY,
    duration INTEGER NOT NULL,
    FOREIGN KEY (ItemID) REFERENCES Item (ItemID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [21]:
%%sql
CREATE TABLE IF NOT EXISTS OnlineItem (
    ItemID INTEGER PRIMARY KEY,
    url TEXT NOT NULL,
    FOREIGN KEY (ItemID) REFERENCES Item (ItemID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [25]:
%%sql
CREATE TABLE IF NOT EXISTS Loan (
    LoanID INTEGER PRIMARY KEY AUTOINCREMENT,
    due DATE NOT NULL,
    userID INTEGER NOT NULL,
    ItemID INTEGER NOT NULL,
    FOREIGN KEY (userID) REFERENCES User (userID) ON DELETE CASCADE,
    FOREIGN KEY (ItemID) REFERENCES Item (ItemID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [27]:
%%sql
CREATE TABLE IF NOT EXISTS Borrows (
    userID INTEGER NOT NULL,
    LoanID INTEGER NOT NULL,
    PRIMARY KEY (userID, LoanID),
    FOREIGN KEY (userID) REFERENCES User (userID) ON DELETE CASCADE,
    FOREIGN KEY (LoanID) REFERENCES Loan (LoanID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [35]:
%%sql 
CREATE TABLE IF NOT EXISTS Fine (
    LoanID INTEGER PRIMARY KEY,
    amount DECIMAL(10,2) NOT NULL,
    status TEXT NOT NULL,
    FOREIGN KEY (LoanID) REFERENCES Loan (LoanID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [37]:
%%sql 
CREATE TABLE IF NOT EXISTS Event (
    EventID INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT NOT NULL
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [41]:
%%sql
CREATE TABLE IF NOT EXISTS Attends (
    userID INTEGER NOT NULL,
    EventID INTEGER NOT NULL,
    PRIMARY KEY (userID, EventID),
    FOREIGN KEY (userID) REFERENCES User (userID) ON DELETE CASCADE,
    FOREIGN KEY (EventID) REFERENCES Event (EventID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [43]:
%%sql
CREATE TABLE IF NOT EXISTS Room (
    roomID INTEGER PRIMARY KEY AUTOINCREMENT,
    capacity INTEGER NOT NULL
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [47]:
%%sql
CREATE TABLE IF NOT EXISTS Uses (
    EventID INTEGER NOT NULL,
    roomID INTEGER NOT NULL,
    PRIMARY KEY (EventID, roomID),
    FOREIGN KEY (EventID) REFERENCES Event (EventID) ON DELETE CASCADE,
    FOREIGN KEY (roomID) REFERENCES Room (roomID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [58]:
%%sql
CREATE TABLE IF NOT EXISTS Personnel (
    staffID INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    position TEXT NOT NULL,
    salary DECIMAL(10,2) NOT NULL
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [62]:
%%sql
CREATE TABLE IF NOT EXISTS Manages (
    staffID INTEGER NOT NULL,
    EventID INTEGER NOT NULL,
    PRIMARY KEY (staffID, EventID),
    FOREIGN KEY (staffID) REFERENCES Personnel (staffID) ON DELETE CASCADE,
    FOREIGN KEY (EventID) REFERENCES Event (EventID) ON DELETE CASCADE
);


Running query in 'sqlite:///library.db'

++
||
++
++

In [64]:
%%sql 
SELECT name FROM sqlite_master WHERE type='table';


Running query in 'sqlite:///library.db'

name
User
sqlite_sequence
Item
CD
OnlineItem
Loan
Borrows
Fine
Event
Attends


In [69]:
%%sql
CREATE TRIGGER update_item_status_on_borrow
AFTER INSERT ON Loan
FOR EACH ROW
BEGIN
    UPDATE Item
    SET status = 'borrowed'
    WHERE ItemID = NEW.ItemID;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [71]:
%%sql
CREATE TRIGGER update_item_status_on_return
AFTER DELETE ON Loan
FOR EACH ROW
BEGIN
    UPDATE Item
    SET status = 'available'
    WHERE ItemID = OLD.ItemID;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [67]:
%%sql
CREATE TRIGGER prevent_borrow_if_unavailable
BEFORE INSERT ON Loan
FOR EACH ROW
BEGIN
    SELECT 
    CASE
        WHEN (SELECT status FROM Item WHERE ItemID = NEW.ItemID) = 'borrowed'
        THEN RAISE(ABORT, 'This item is already borrowed.')
    END;
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [29]:
%%sql

CREATE TRIGGER generate_fine_on_late_return
AFTER DELETE ON Loan
FOR EACH ROW
WHEN julianday('now') > julianday(OLD.due)
BEGIN
    INSERT INTO Fine (LoanID, amount, status)
    VALUES (OLD.LoanID, 1.00 * (julianday('now') - julianday(OLD.due)), 'Unpaid');
END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [31]:
%%sql
SELECT name FROM sqlite_master WHERE type = 'trigger';


Running query in 'sqlite:///library.db'

name
prevent_borrow_if_unavailable
update_item_status_on_borrow
update_item_status_on_return
generate_fine_on_late_return


In [ ]:
%%sql
INSERT INTO User (UserID, Email, Preference) 
VALUES 
    (1, 'alice.brown@example.com'),
    (2,  'bob.smith@example.com'),
    (3, 'carol.jones@example.com'),
    (4, 'dave.wilson@example.com'),
    (5, 'eve.davis@example.com'),
    (6,  'frank.miller@example.com'),
    (7,  'grace.lee@example.com'),
    (8,'henry.taylor@example.com'),
    (9, 'ivy.anderson@example.com'),
    (10,  'jack.thomas@example.com');


In [ ]:
%%sql
INSERT INTO Item (ItemID, Title, Type, Author, Status) 
VALUES 
    (101, 'The Great Gatsby', 'Book', 'F. Scott Fitzgerald', 'Available'),
    (102, '1984', 'Book', 'George Orwell', 'Borrowed'),
    (103, 'National Geographic', 'Magazine', 'Various', 'Available'),
    (104, 'Time', 'Magazine', 'Various', 'Reserved'),
    (105, 'Thriller', 'CD', 'Michael Jackson', 'Available'),
    (106, 'Inception', 'DVD', 'Christopher Nolan', 'Available'),
    (107, 'Learn Python', 'Online Book', 'John Doe', 'Available'),
    (108, 'Interstellar Soundtrack', 'CD', 'Hans Zimmer', 'Borrowed'),
    (109, 'The New Yorker', 'Magazine', 'Various', 'Available'),
    (110, 'Science Journal', 'Scientific Journal', 'Various', 'Available');


In [ ]:
%%sql 
INSERT INTO Loan (LoanID, UserID, ItemID, DueDate, Status) 
VALUES 
    (201, 1, 102, '2025-04-15', 'Borrowed'),
    (202, 2, 105, '2025-04-20', 'Returned'),
    (203, 3, 108, '2025-04-18', 'Borrowed'),
    (204, 4, 101, '2025-04-12', 'Returned'),
    (205, 5, 107, '2025-04-22', 'Borrowed'),
    (206, 6, 106, '2025-04-16', 'Returned'),
    (207, 7, 104, '2025-04-19', 'Borrowed'),
    (208, 8, 103, '2025-04-17', 'Returned'),
    (209, 9, 110, '2025-04-21', 'Borrowed'),
    (210, 10, 109, '2025-04-25', 'Returned');


In [ ]:
%%sql
INSERT INTO Fine (FineID, LoanID, Amount, Status) 
VALUES 
    (301, 201, 10.00, 'Unpaid'),
    (302, 202, 0.00, 'Paid'),
    (303, 203, 5.00, 'Unpaid'),
    (304, 204, 0.00, 'Paid'),
    (305, 205, 0.00, 'Paid'),
    (306, 206, 0.00, 'Paid'),
    (307, 207, 3.50, 'Unpaid'),
    (308, 208, 0.00, 'Paid'),
    (309, 209, 8.00, 'Unpaid'),
    (310, 210, 0.00, 'Paid');


In [ ]:
%%sql
INSERT INTO Event (EventID, Type, RoomID) 
VALUES 
    (401, 'Book Club', 1),
    (402, 'Film Screening', 2),
    (403, 'Art Show', 3),
    (404, 'Tech Talk', 4),
    (405, 'Cooking Class', 5),
    (406, 'History Lecture', 2),
    (407, 'Science Seminar', 3),
    (408, 'Music Concert', 1),
    (409, 'Poetry Reading', 4),
    (410, 'Children Storytime', 5);


In [ ]:
%%sql
INSERT INTO Room (RoomID, Capacity) 
VALUES 
    (1, 30),
    (2, 50),
    (3, 40),
    (4, 25),
    (5, 20),
    (6, 60),
    (7, 15),
    (8, 35),
    (9, 45),
    (10, 55);


In [ ]:
%%sql 
INSERT INTO Personnel (StaffID, Name, Position, Salary) 
VALUES 
    (501, 'Alice Johnson', 'Librarian', 45000),
    (502, 'Bob Williams', 'Event Manager', 40000),
    (503, 'Carol King', 'Assistant Librarian', 35000),
    (504, 'David Smith', 'Archivist', 38000),
    (505, 'Eva Brown', 'Digital Resource Manager', 42000),
    (506, 'Frank Davis', 'Security', 30000),
    (507, 'Grace Lee', 'Custodian', 28000),
    (508, 'Henry Taylor', 'Technician', 32000),
    (509, 'Ivy Anderson', 'Event Coordinator', 41000),
    (510, 'Jack Thomas', 'Manager', 50000);
